In [8]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

load_dotenv()

spark = SparkSession.builder.appName("RealEstate_Avito_Docker").getOrCreate()

storage_account = "strealestatehamza"
container = "realestate"

adls_key = os.getenv("ADLS_ACCOUNT_KEY")
if not adls_key:
    raise RuntimeError("ADLS_ACCOUNT_KEY missing from .env")

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    adls_key,
)
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    adls_key,
)

In [9]:
# ========================================
# CELLULE 1 : Imports et Configuration
# ========================================
import re
import csv
import json
import time
import random
from typing import List, Dict, Any
from urllib.parse import urljoin
from datetime import datetime

import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.avito.ma"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}

In [10]:
# ========================================
# CELLULE 2 : Fonctions basiques
# ========================================

def fetch_html(url: str) -> str:
    """Télécharge le HTML d'une page Avito."""
    resp = requests.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    return resp.text


def extract_id_from_url(url: str) -> str:
    """Extrait l'ID numérique à la fin de l'URL Avito."""
    m = re.search(r"_([0-9]+)\.htm$", url.split("?")[0])
    return m.group(1) if m else ""


In [11]:
# ========================================
# CELLULE 3 : breadcrumbs, catégorie, localisation & date
# ========================================

def get_breadcrumbs(soup: BeautifulSoup) -> Dict[str, Any]:
    """Retourne breadcrumbs_list et breadcrumbs (string)"""
    crumbs = []
    ol = soup.find("ol", class_=re.compile(r"sc-16q833i-0"))
    if ol:
        for li in ol.find_all("li", class_=re.compile(r"sc-16q833i-3")):
            span_or_a = li.find(["a", "span"])
            if span_or_a:
                text = span_or_a.get_text(strip=True)
                if text:
                    crumbs.append(text)

    return {
        "breadcrumbs_list": crumbs,
        "breadcrumbs": " > ".join(crumbs) if crumbs else "",
    }


def get_category_label(soup: BeautifulSoup) -> str:
    """
    Récupère le libellé de catégorie du bloc 'Categorie' (ex: 'Appartements, à louer').
    """
    cat_section = soup.find("div", attrs={"aria-label": re.compile(r"Category ")})
    if not cat_section:
        return ""

    texts = cat_section.stripped_strings
    for t in texts:
        if "Categorie" in t:
            continue
        return t
    return ""


def get_location_and_date(soup: BeautifulSoup) -> Dict[str, str]:
    """
    Récupère location et published_date_raw.
    Ajoute également scraping_time comme Mubawab.
    """
    location = ""
    published_date_raw = ""

    # Date de publication depuis balise <time>
    time_tag = soup.find("time", attrs={"datetime": True})
    if time_tag:
        published_date_raw = time_tag["datetime"]

    # Location (ex: Racine, Casablanca)
    location_span = None
    for svg in soup.find_all("svg", title=re.compile(r"MapPinFill Icon")):
        parent = svg.parent
        location_span = parent.find("span", class_=re.compile(r"sc-16573058-17"))
        if location_span:
            break

    if location_span:
        location = location_span.get_text(strip=True)

    # Temps de scraping (comme Mubawab)
    scraping_time = datetime.utcnow()
    scraping_time_iso = scraping_time.isoformat()

    return {
        "location": location,
        "published_date": published_date_raw,  # Harmonisation avec Mubawab
        "scraping_time": scraping_time_iso,
    }


In [12]:
# ========================================
# CELLULE 4 : titre, prix, description, images
# ========================================

def get_title_and_price(soup: BeautifulSoup) -> Dict[str, str]:
    """Récupère le titre (h1) et le prix."""
    title = ""
    price_text = ""

    # Titre
    h1 = soup.find("h1")
    if h1:
        title = h1.get_text(strip=True)

    # Prix
    price_block = soup.find("div", class_=re.compile(r"sc-16573058-10"))
    if price_block:
        p = price_block.find("p")
        if p:
            price_text = p.get_text(strip=True)
    else:
        p = soup.find(string=re.compile(r"DH"))
        if p:
            price_text = p.strip()

    return {
        "title": title,
        "price_text": price_text,
    }


def get_description(soup: BeautifulSoup) -> str:
    """Récupère la description depuis le bloc Description."""
    desc_container = None
    for div in soup.find_all("div", class_=re.compile(r"sc-b59a33d2-3")):
        h2 = div.find("h2")
        if h2 and "Description" in h2.get_text():
            desc_container = div
            break

    if desc_container:
        text_parts = []
        for node in desc_container.find_all(["p", "div", "span"], recursive=True):
            t = node.get_text(" ", strip=True)
            if t:
                text_parts.append(t)
        description = " ".join(text_parts)
        description = re.sub(r"\s+", " ", description).strip()
        return description

    # fallback
    p = soup.find("p")
    if p:
        return p.get_text(" ", strip=True)
    return ""


def get_images(soup: BeautifulSoup) -> List[str]:
    """Récupère toutes les URLs d'images de la galerie principale."""
    urls = []
    for img in soup.select("div.picture img"):
        src = img.get("src")
        if src and "content.avito.ma/classifieds/images" in src:
            if src not in urls:
                urls.append(src)
    return urls


In [13]:
# ========================================
# CELLULE 5 : attributs, équipements, infos vendeur
# ========================================

def get_attributes_and_equipments(soup: BeautifulSoup) -> Dict[str, Any]:
    """
    Récupère :
      - attributes: dict { "Chambres": "3", "Salle de bain": "2", ... }
      - equipments: liste ["Ascenseur", "Balcon", ...]
    """
    attributes = {}
    equipments = []

    attr_blocks = soup.find_all("div", class_=re.compile(r"sc-cd1c365e-0"))
    for block in attr_blocks:
        parent = block.find_parent("div", class_=re.compile(r"sc-b59a33d2-3"))
        is_equip_section = False
        if parent:
            h2 = parent.find("h2")
            if h2 and "Équipements" in h2.get_text():
                is_equip_section = True

        for item in block.find_all("div", class_=re.compile(r"sc-cd1c365e-1")):
            img = item.find("img", alt=True)
            label_from_alt = img["alt"].strip() if img else ""

            value_span = item.find("span", class_=re.compile(r"fjZBup"))
            value = value_span.get_text(strip=True) if value_span else ""

            if is_equip_section:
                if label_from_alt:
                    equipments.append(label_from_alt)
                elif value:
                    equipments.append(value)
            else:
                if label_from_alt and value:
                    attributes[label_from_alt] = value

    return {
        "attributes": attributes,
        "equipments": equipments,
    }


def get_seller_info(soup: BeautifulSoup) -> Dict[str, Any]:
    """Récupère seller_name, seller_url, seller_is_store."""
    seller_name = ""
    seller_url = ""
    seller_is_store = False

    seller_block = soup.find("div", attrs={"data-test": "av_sellerInfo"})
    if not seller_block:
        seller_block = soup.find("div", class_=re.compile(r"sc-1l0do2b-0"))

    if seller_block:
        a = seller_block.find("a", href=True)
        if a:
            seller_url = urljoin(BASE_URL, a["href"])
            name_tag = a.find("p") or a.find("span")
            if name_tag:
                seller_name = name_tag.get_text(strip=True)

        text_block = seller_block.get_text(" ", strip=True)
        if "Voir la boutique" in text_block:
            seller_is_store = True

        if seller_block.find("title", string=re.compile(r"Store Icon")):
            seller_is_store = True

    return {
        "seller_name": seller_name,
        "seller_url": seller_url,
        "seller_is_store": seller_is_store,
    }

In [14]:
# ========================================
# CELLULE 6 : Parser principal
# ========================================

def parse_avito_ad(url: str) -> Dict[str, Any]:
    """Scrape une annonce Avito et retourne un dict."""
    html = fetch_html(url)
    soup = BeautifulSoup(html, "html.parser")

    ad_id = extract_id_from_url(url)

    crumbs = get_breadcrumbs(soup)
    cat_label = get_category_label(soup)
    loc_date = get_location_and_date(soup)
    title_price = get_title_and_price(soup)
    description = get_description(soup)
    images = get_images(soup)
    attrs_equip = get_attributes_and_equipments(soup)
    seller = get_seller_info(soup)

    ad_data: Dict[str, Any] = {
        "id": ad_id,
        "url": url,
        "title": title_price["title"],
        "price_text": title_price["price_text"],
        "location": loc_date["location"],
        "published_date": loc_date["published_date"],
        "scraping_time": loc_date["scraping_time"],
        "breadcrumbs_list": crumbs["breadcrumbs_list"],
        "breadcrumbs": crumbs["breadcrumbs"],
        "category_label": cat_label,
        "description": description,
        "images": images,
        "attributes": attrs_equip["attributes"],
        "equipments": attrs_equip["equipments"],
        "seller_name": seller["seller_name"],
        "seller_url": seller["seller_url"],
        "seller_is_store": seller["seller_is_store"],
    }

    return ad_data


In [15]:
# ========================================
# CELLULE 7 : extraire URLs depuis listing
# ========================================

def get_ad_urls_from_listing(listing_url: str) -> List[str]:
    """Récupère toutes les URLs d'annonces d'une page listing Avito."""
    html = fetch_html(listing_url)
    soup = BeautifulSoup(html, "html.parser")

    ad_urls = set()

    for a in soup.find_all("a", href=True):
        href = a["href"]

        if href.startswith("/"):
            href = urljoin(BASE_URL, href)

        if "avito.ma" not in href:
            continue

        if re.search(r"_[0-9]+\.htm$", href):
            clean = href.split("?")[0]
            ad_urls.add(clean)

    ad_urls = sorted(ad_urls)
    print(f"🌐 Trouvé {len(ad_urls)} annonces sur {listing_url}")
    return ad_urls



In [16]:
# ========================================
# CELLULE 8 : Save CSV (optionnel)
# ========================================

def save_ads_to_csv(ad_dicts: List[Dict[str, Any]], filename: str) -> None:
    """Sauvegarde une liste d'annonces (dict) dans un fichier CSV."""
    if not ad_dicts:
        print(f"Aucune annonce à sauvegarder pour {filename}")
        return

    fieldnames = [
        "id",
        "url",
        "title",
        "price_text",
        "location",
        "published_date",
        "scraping_time",
        "breadcrumbs",
        "breadcrumbs_list",
        "category_label",
        "description",
        "attributes",
        "equipments",
        "seller_name",
        "seller_url",
        "seller_is_store",
    ]

    with open(filename, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for ad in ad_dicts:
            row: Dict[str, Any] = {}

            for key in fieldnames:
                if key in ("attributes", "equipments", "breadcrumbs_list"):
                    continue
                row[key] = ad.get(key, "")

            row["attributes"] = json.dumps(
                ad.get("attributes", {}), ensure_ascii=False
            )
            row["equipments"] = "; ".join(ad.get("equipments", []))
            row["breadcrumbs_list"] = json.dumps(
                ad.get("breadcrumbs_list", []), ensure_ascii=False
            )

            writer.writerow(row)

    print(f"✅ Sauvegardé {len(ad_dicts)} annonces dans {filename}")



In [17]:
# ========================================
# CELLULE 9 : scraper listing -> liste
# ========================================

def scrape_listing_to_list(listing_url: str) -> List[Dict[str, Any]]:
    """Scrape la 1ère page d'un listing Avito et retourne une liste de dicts."""
    ad_urls = get_ad_urls_from_listing(listing_url)

    all_ads: List[Dict[str, Any]] = []
    total = len(ad_urls)

    for i, ad_url in enumerate(ad_urls, 1):
        print(f"[{i}/{total}] Scraping {ad_url}")
        try:
            ad_data = parse_avito_ad(ad_url)
            all_ads.append(ad_data)
        except Exception as e:
            print(f"  ❌ Erreur sur {ad_url}: {e}")

        time.sleep(random.uniform(1.0, 2.5))

    return all_ads

In [18]:
# ========================================
# CELLULE 10 : Workflow Spark + ADLS (Avito, PARQUET)
# ========================================
from datetime import datetime
from pyspark.sql import functions as F
import json

# 🔒 Variables stockage ADLS
storage_account = "strealestatehamza"
container = "realestate"

# --- 1. URLs des listings Avito ---
ventes_url = "https://www.avito.ma/fr/maroc/ventes_immobilieres-à_vendre"
locations_url = "https://www.avito.ma/fr/maroc/locations_immobilieres-à_louer"

# --- 2. Scraping -> listes de dicts ---
raw_ventes_ads = scrape_listing_to_list(ventes_url)
raw_locations_ads = scrape_listing_to_list(locations_url)

print(f"Ventes récupérées: {len(raw_ventes_ads)}")
print(f"Locations récupérées: {len(raw_locations_ads)}")


def normalize_ads_for_spark(ads):
    """Convertit les champs dict/list en strings pour Spark (OK aussi pour Parquet)."""
    normalized = []
    for ad in ads:
        d = ad.copy()
        d["attributes"] = json.dumps(d.get("attributes", {}), ensure_ascii=False)
        d["equipments"] = "; ".join(d.get("equipments", []) or [])
        d["breadcrumbs_list"] = json.dumps(d.get("breadcrumbs_list", []) or [], ensure_ascii=False)
        d["images"] = ", ".join(d.get("images", []) or [])
        normalized.append(d)
    return normalized


ventes_ads = normalize_ads_for_spark(raw_ventes_ads)
locations_ads = normalize_ads_for_spark(raw_locations_ads)

# --- 3. Conversion en DataFrame Spark ---
ventes_df = spark.createDataFrame(ventes_ads)
locations_df = spark.createDataFrame(locations_ads)

now = datetime.utcnow().isoformat()

# Colonnes techniques
ventes_df = (
    ventes_df
    .withColumn("source_site", F.lit("avito"))
    .withColumn("offre", F.lit("vente"))
    .withColumn("ingest_ts", F.lit(now))
)

locations_df = (
    locations_df
    .withColumn("source_site", F.lit("avito"))
    .withColumn("offre", F.lit("location"))
    .withColumn("ingest_ts", F.lit(now))
)

# --- Debug schemas ---
print("📌 Schéma ventes")
ventes_df.printSchema()
print("📌 Schéma locations")
locations_df.printSchema()

# --- 4. Chemins RAW dans ADLS ---
date_path = datetime.utcnow().strftime("%Y/%m/%d/%H%M%S")
base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/raw"

ventes_path = f"{base_path}/avito/ventes/{date_path}"
locations_path = f"{base_path}/avito/locations/{date_path}"

# --- 5. Écriture en PARQUET (❌ plus de CSV) ---
(
    ventes_df
    .coalesce(1)
    .write
    .mode("overwrite")
    .parquet(ventes_path)
)

(
    locations_df
    .coalesce(1)
    .write
    .mode("overwrite")
    .parquet(locations_path)
)

print("✅ Ventes Avito (PARQUET) ->", ventes_path)
print("✅ Locations Avito (PARQUET) ->", locations_path)

🌐 Trouvé 38 annonces sur https://www.avito.ma/fr/maroc/ventes_immobilieres-à_vendre
[1/38] Scraping https://www.avito.ma/fr/agdal/terrains_et_fermes/Terrain_bijoux_R5_AGDAL_53485705.htm
[2/38] Scraping https://www.avito.ma/fr/aouama_gharbia/maisons/maison_semi_fini_à_vendre_à_lot_sanaoubari_56970217.htm
[3/38] Scraping https://www.avito.ma/fr/autre_secteur/terrains_et_fermes/Terrain_agricole_équipé_à_Benimadane_à_vendre_ou_43988341.htm
[4/38] Scraping https://www.avito.ma/fr/autre_secteur/villas_et_riads/Villa_neuve_à_vendre_670_m²_à_Temara_54973482.htm
[5/38] Scraping https://www.avito.ma/fr/beauséjour/appartements/Appartement_à_vendre_115_m²_à_Casablanca_55194354.htm
[6/38] Scraping https://www.avito.ma/fr/bir_rami/appartements/Appartement_à_vendre_65_m²_à_Kénitra_57164756.htm
[7/38] Scraping https://www.avito.ma/fr/bouskoura_centre/terrains_et_fermes/Terrain_à_vendre_bouskoura_54038488.htm
[8/38] Scraping https://www.avito.ma/fr/branes_2/appartements/Appartement_à_vendre_57133042.ht